In [ ]:
!wget https://raw.githubusercontent.com/tproffen/SmartRobotCamp/master/Notebooks/Jetbot/ObjectFollow/mscoco_complete_label_map.json

## Follow an object

This is very simular to the object detection. Look at the comments to see the additional code. We need to select an object type to follow. To figure out the index, check you [this file](https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbtxt). 

In [ ]:
from jetbot import ObjectDetector

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

In [ ]:
from jetbot import Camera
from jetbot import bgr8_to_jpeg
from jetbot import Robot

camera = Camera.instance(width=300, height=300)
robot = Robot()

We create some more widgets to control speeds and to set the central object to follow.

In [ ]:
from IPython.display import display
import ipywidgets.widgets as widgets

image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.IntText(value=47, description='tracked label')
cx_widget = widgets.FloatText(value=0, description='cx')
cy_widget = widgets.FloatText(value=0, description='cy')
width_widget = widgets.FloatText(value=0, description='width')

tolerance_widget = widgets.FloatSlider(description='tolerance', min=0.05, max=0.5, value=0.10, step=0.01)
speed_widget = widgets.FloatSlider(description='speed', min=0.05, max=0.5, value=0.10, step=0.01)

display(image_widget, label_widget, cx_widget, cy_widget, width_widget, tolerance_widget, speed_widget)

In [ ]:
import json

descriptions=[]

with open('mscoco_complete_label_map.json') as json_file:
    data = json.load(json_file)
    for p in data['items']:
        descriptions.append(p['display_name'])

Additional code below is added to find the center of the object to follow and use that to determine how to turn the robot, so it keeps looking at it.

In [ ]:
import cv2
import numpy as np

width = int(image_widget.width)
height = int(image_widget.height)

def drawBoundingBox(imgcv,box,label,color):
    x1,y1,x2,y2 = (int(width * box[0]), int(height * box[1]), int(width * box[2]), int(height * box[3]))
    cv2.rectangle(imgcv,(x1,y1),(x2,y2),color,2)
    labelSize=cv2.getTextSize(label,cv2.FONT_HERSHEY_COMPLEX,0.5,2)
    _x1 = x1
    _y1 = y1
    _x2 = x1+labelSize[0][0]
    _y2 = y1-int(labelSize[0][1])
    cv2.rectangle(imgcv,(_x1,_y1),(_x2,_y2),color,cv2.FILLED)
    cv2.putText(imgcv,label,(x1,y1),cv2.FONT_HERSHEY_COMPLEX,0.5,(255,255,255),1)
    return imgcv
    
def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Finds the detection closest to the image center"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection

def execute(change):
    image = change['new']
    
    # compute all detected objects
    detections = model(image)
    
    # draw all detections on image
    
    for det in detections[0]:
        image=drawBoundingBox(image,det['bbox'],descriptions[det['label']],(255,0,0))

    # select detections that match selected class label
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
    
    # get detection closest to center of field of view and draw it
    det = closest_detection(matching_detections)
    if det is not None:
        image=drawBoundingBox(image,det['bbox'],descriptions[det['label']],(0,0,255))
        center = detection_center(det)
        width = det['bbox'][2] - det['bbox'][0]
        cx_widget.value=center[0]
        cy_widget.value=center[1]
        width_widget.value=width
        
        limit=float(tolerance_widget.value)
        speed=float(speed_widget.value)

        # turn the robot
        if center[0] > limit:
            pass
            robot.set_motors( speed,-speed)
        elif center[0] < -limit:
            pass
            robot.set_motors(-speed, speed)
        else:
            pass
            robot.stop()
    else:
        pass
        robot.stop()
    
    # update image widget
    image_widget.value = bgr8_to_jpeg(image)
    
execute({'new': camera.value})

This again will run the code for every camera frame.

In [ ]:
camera.unobserve_all()
camera.observe(execute, names='value')

Run the cell below to stop following the object :)

In [ ]:
camera.unobserve_all()
robot.stop()